In [9]:
import os 
import re 
import csv 
from docx import Document 
from PyPDF2 import PdfReader 
import win32com.client as win32 
from win32com.client import constants
import glob

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return '\n'.join(paragraphs)

def save_as_docx(file_path):
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(file_path)
    doc.Activate()

    # Rename path with .docx
    new_file_abs = os.path.abspath(file_path)
    new_file_abs = os.path.splitext(file_path)[0] + '.docx'

    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)

def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)  # Remove extra space
    return cleaned_text

# Database path:
input_directory = r"D:\Projects\Classification Project\Resumes"
output_csv = 'output2.csv'

with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Text'])  # Write header row

    for file_name in os.listdir(input_directory):
        file_path = os.path.join(input_directory, file_name)
        if file_name.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
        elif file_name.lower().endswith('.docx'):
            text = extract_text_from_docx(file_path)
        elif file_name.lower().endswith('.doc'):
            save_as_docx(file_path)
            text = extract_text_from_docx(file_path + 'x')
            os.remove(file_path + 'x')  # Remove the temporary .docx file
        else:
            print(f"Skipping file: {file_name}. Unsupported file format.")
            continue
        
        cleaned_text = clean_text(text)
        writer.writerow([cleaned_text])

print("Extraction and cleaning complete. Saved to output2.csv.")

ModuleNotFoundError: ignored